In [2]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("Colorful/RTA")
model_path = "/kaggle/input/modelseveritypred/modelPrioritySeverity"
model = AutoModelForSequenceClassification.from_pretrained(model_path)

model.to(device)  

tokenizer_config.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [4]:
csv_path = '/kaggle/input/severityprediction/severity_test.csv'
df = pd.read_csv(csv_path)

label_map = {
    "blocker": 0,
    "critical": 1,
    "major": 2,
    "minor": 3,
    "trivial": 4
}


df['label'] = df['label'].map(label_map)
df.head()

,sentence1,label
0,[indentation]Auto comment doesn't correctly al...,4
1,Debugging JComboBox with Java 6 kills X input....,0
2,Standard output in Output window is not flushe...,0
3,JDBC connection does not interpret proxy excep...,0
4,NullPointerException at java.lang.Class.isAssi...,0


In [5]:
true_labels = []
predicted_labels = []

for index, row in df.iterrows():
    sentence = row['sentence1']
    true_label = row['label']
    
    # Tokenize input sentence
    tokenized_inputs = tokenizer(sentence, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
    
    inputs = {key: value.to(device) for key, value in tokenized_inputs.items()}
    
    with torch.no_grad():
        logits = model(**inputs).logits
        probabilities = torch.softmax(logits, dim=1)

        prediction_label = torch.argmax(probabilities, dim=1).cpu().item()
        
        true_labels.append(true_label)
        predicted_labels.append(prediction_label)

# Calculate evaluation metrics
accuracy = accuracy_score(true_labels, predicted_labels) * 100
precision = precision_score(true_labels, predicted_labels, average='weighted') * 100
recall = recall_score(true_labels, predicted_labels, average='weighted') * 100

print(f"Accuracy: {accuracy:.4f}%")
print(f"Precision: {precision:.4f}%")
print(f"Recall: {recall:.4f}%")

Accuracy: 65.5899%
Precision: 66.2795%
Recall: 65.5899%
